In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

import sklearn
from sklearn.svm import LinearSVC
from sklearn import preprocessing

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Feature Extraction

In [ ]:
# pulling original VGG16
model = models.vgg16(weights = 'DEFAULT')

# new class specific to "feature extraction" with no classifier
class FeatureExtractor(nn.Module):
    def __init__(self, layers=1):
      super().__init__()
      self.layers = layers
      self.features = model.features[:self.layers + 1]

    def forward(self, x):
      x = self.features(x)
      return x

Load data

In [ ]:
# Define the same transformations used when the data was prepared (important!)
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

image_data_root = '/content/drive/My Drive/Frechette Lab/train_dataset'

try:
    train_dataset = datasets.ImageFolder(root=image_data_root, transform=transform)
    print(f"Found {len(train_dataset)} images belonging to {len(train_dataset.classes)} classes.")

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    print("DataLoader created successfully.")

except Exception as e:
    print(f"Error creating ImageFolder dataset: {e}")
    print("Please ensure the directory structure in")
    print(f"{image_data_root} follows the ImageFolder convention (subdirectories for classes).")
    print("Also check file permissions and that the directory exists.")

Binarize labels

In [ ]:
lb = preprocessing.LabelBinarizer(neg_label = -1, pos_label = 1)
lb.fit(labels)

Classifier

In [ ]:
# classifier
clf = LinearSVC(loss = 'hinge')
